In [5]:
import pandas as pd
import torch
from transformers import BertTokenizer, AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader
import tools
from utils.news_set import NewsSet
from models.news_classifier import NewsClassifier

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
labels=['时尚','家居','教育','股票','娱乐','彩票','社会','房产','星座','科技','财经','时政','游戏','体育']
data_set = pd.read_csv('data/news_data1.csv')
data_set['label']=data_set['label'].apply(lambda x: labels.index(x))
data_set['label'] = data_set['label'].astype('int64')
data_set['text'] = data_set['text'].astype('string')

In [7]:
bert_path = 'models/bert_pretrained/bert-base-chinese/'
tokenizer = BertTokenizer.from_pretrained(bert_path)
model = NewsClassifier(bert_path, 14).to(DEVICE)

Some weights of the model checkpoint at models/bert_pretrained/bert-base-chinese/ were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
EPOCHS = 1
train_set, val_set = tools.train_val_split(data_set)
train_loader = DataLoader(NewsSet(train_set), batch_size=32, shuffle=True)
print(len(train_loader))
val_loader = DataLoader(NewsSet(val_set), batch_size=32, shuffle=True)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=len(train_loader),
                                            num_training_steps=EPOCHS*len(train_loader))

tools.train(model, train_loader, val_loader, tokenizer, optimizer, scheduler, DEVICE, EPOCHS)
torch.save(model, 'models/news_model.pkl')

1666
***** Training *****


KeyboardInterrupt: 